In [ ]:
import requests

from obspy import read, UTCDateTime
from obspy.taup import TauPyModel
from obspy.geodetics import locations2degrees
from obspy import read, UTCDateTime

In [ ]:
# We use the iasp91 reference model
TAUP_MODEL = TauPyModel(model="iasp91")
FDSN_DATASELECT = "http://rdsa.knmi.nl/fdsnws//dataselect/1/query"
FDSN_EVENT = "http://knmi-sproc-l01p.knmi.ssc-campus.nl:8080/fdsnws/event/1/query"
FDSN_STATION = "http://rdsa.knmi.nl/fdsnws/station/1/query"
MAX_RADIUS = 20

In [ ]:
# Define class for Events
class Event:
    def __init__(self, line):
        self.id, self.time, self.latitude, self.longitude, self.depth = line.split("|")[
            :5
        ]
        self.latitude = float(self.latitude)
        self.longitude = float(self.longitude)
        self.depth = float(self.depth)


def getEvent(identifier):
    # We query for a single event identifier and request a text format return
    queryString = "&".join(["eventid=%s" % identifier, "format=text"])

    # Create the query for an event identifier
    r = requests.get("%s?%s" % (FDSN_EVENT, queryString))
    
    print(r.history[0].url if r.history else r.url)

    # Split by lines and remove head & tail
    lines = r.text.split("\n")[1:-1]

    # Return Event classes for each entry
    return list(map(Event, lines))[0]


In [ ]:
class Station:
    def __init__(self, line):
        self.network, self.station, self.latitude, self.longitude = line.split("|")[:4]
        self.latitude = float(self.latitude)
        self.longitude = float(self.longitude)


def getStations(event, net=None, stat=None, channel="*"):
    # We query with the event location and a maximum radius around the event
    if net and stat:
        queryString = "&".join(["network=%s" % net, "station=%s" % stat, "format=text"])
    else:
        queryString = "&".join(
            [
                "latitude=%s" % event.latitude,
                "longitude=%s" % event.longitude,
                "maxradius=%s" % MAX_RADIUS,
                "channel=%s" % channel,
                "format=text",
            ]
        )

    # Request from webservice
    r = requests.get("%s?%s" % (FDSN_STATION, queryString))
    
    print(r.history[0].url if r.history else r.url)

    # Split by lines and remove head & tail
    lines = r.text.split("\n")[1:-1]

    # Return Event classes for each entry
    return map(Station, lines)


In [ ]:
def getPArrival(event, station):
    # Determine the arc distance using the haversine formula
    arcDistanceDegrees = locations2degrees(
        event.latitude, station.latitude, event.longitude, station.longitude
    )

    # Calculate the theoretical P-arrival time
    arrivals = TAUP_MODEL.get_travel_times(
        source_depth_in_km=1e-3 * event.depth,
        distance_in_degree=arcDistanceDegrees,
        phase_list=["P"],
    )

    # Add the theorical P-arrival delta to the event time
    return UTCDateTime(event.time) + arrivals[0].time


In [ ]:
def plot_event(event_id, net=None, stat=None, channel="*"):
    event = getEvent(event_id)
    print(f"Found event {event=}")
    # Go over all stations returned in the radius
    stations = getStations(event, net, stat, channel)

    for idx, station in enumerate(stations):

        # Get the theoretical (TauP) pArrval from event to station
        stationArrivalTime = getPArrival(event, station)

        # Create the query for fdsn-dataselect
        # between 300 seconds before & 1200 seconds after the theoretical P-arrival
        queryString = "&".join(
            [
                "network=%s" % station.network,
                "station=%s" % station.station,
                "starttime=%s" % (stationArrivalTime - 300).isoformat(),
                "endtime=%s" % (stationArrivalTime + 1200).isoformat(),
                "channel=%s" % channel,
            ]
        )

        # Get the waveform data and read to ObsPy Stream
        # Empty responses are skipped
        try:
            url = "%s?%s" % (FDSN_DATASELECT, queryString)
            print(f"Getting data using {url=}")
            st = read(url)
        except Exception:
            continue

        # Use with ObsPy and apply a filter, then plot
        # Alternatively, we would save the data to a file
#         st.filter("lowpass", freq=0.5)
        st.plot()

        # Break after the first result
        if idx >= 5:
            break


In [ ]:
# North Korea nuclear blast
plot_event("knmi2017rfxf", "NL", "HGN", "BH?")

In [ ]:
# Sonic boom
plot_event("knmi2021dzln")

In [ ]:
# Induced
plot_event("knmi2021qfyc", "NL", "ARCN", "BH?")

In [ ]:
# Tectonic
plot_event("knmi2021qvoa", "NL", "HRDB", "HG?")

In [ ]:
# Noise
plot_event("knmi2021qfoq", "NL", "DR031", "HH?")

In [ ]:
# Greece
st = read("http://eida.gein.noa.gr/fdsnws/dataselect/1/query?network=HL&station=IACM&start=2021-09-27T00:00:00&end=2021-09-27T23:59:59&channel=HNE")
st.plot(type="dayplot")

In [ ]:
# Switzerland
# https://www.orfeus-eu.org/rrsm/event/20211005_0000063/CH/SZEK/
fds = read("http://eida.ethz.ch/fdsnws/dataselect/1/query?network=CH&station=SZEK&start=2021-10-05T05:39:20&end=2021-10-05T05:40:25")
fds.plot()